<a href="https://colab.research.google.com/github/yu072333/114-/blob/main/%E7%A8%8B%E8%A8%ADHW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# ----------------------------------------------------------------------
# 1. 函式庫安裝與導入
# ----------------------------------------------------------------------
!pip install google-genai gradio pandas gspread -q

import os
import gradio as gr
import pandas as pd
from datetime import datetime
import gspread
from google.colab import auth
from google.auth import default
from google import genai
from google.genai.errors import APIError

# ----------------------------------------------------------------------
# 2. 變數與認證設定
# ----------------------------------------------------------------------

# --- (A) Gemini API 金鑰設定 ---
# 這是最關鍵的步驟！請將 'YOUR_GEMINI_API_KEY' 替換為您的真實金鑰。
# 貼上後請執行此儲存格，讓程式碼讀取您的 Key。
# ----------------------------------------------------
os.environ['GEMINI_API_KEY'] = userdata.get('gemini')

try:
    client = genai.Client()
    GEMINI_MODEL = 'gemini-2.5-flash'
    print("✅ Gemini Client 初始化成功。")
except Exception as e:
    print(f"❌ Gemini Client 初始化失敗: {e}")
    print("請確保您已設定有效的 GEMINI_API_KEY。")


# --- (B) Google Sheets 試算表設定與認證 ---
# 請替換成您自己的 Google 試算表網址！
SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/1Dk1HCgX0yPg3gHSbxZL8bNssT5PaR5NvIRW3K7fd9M4/edit?gid=0#gid=0'

# 記帳資料輸入的工作表名稱 (請根據您的試算表實際名稱調整)
INPUT_SHEET_NAME = '工作表1'
# AI 分析報告輸出到的工作表名稱
OUTPUT_SHEET_NAME = '分析報告'

# 核心欄位名稱 (必須與您的 Google Sheet 第一行的標題完全一致)
COLUMN_NAMES = ['日期', '品項', '金額', '支付方式', '是否必需']


# ----------------------------------------------------
# 3. Google Sheets 認證 (執行後會彈出授權視窗，請完成認證)
# ----------------------------------------------------
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # 打開試算表並連接
    gsheets = gc.open_by_url(SPREADSHEET_URL)
    sheets_enabled = True
    print("✅ Google Sheets 認證與連接成功。")
except Exception as e:
    print(f"❌ Google Sheets 認證或連接失敗: {e}")
    sheets_enabled = False
    gsheets = None


# 定義 Gradio 介面選項
CATEGORIES = ['外食', '買菜', '交通', '購物', '娛樂', '其他']
PAYMENTS = ['電子支付', '信用卡', '現金']

✅ Gemini Client 初始化成功。
✅ Google Sheets 認證與連接成功。


In [38]:
def add_new_expense(date, category, amount, payment_method, is_essential):
    """
    將一筆新的消費記錄寫入 Google 試算表。
    """
    if not sheets_enabled:
         return "❌ Google Sheets 未啟用，無法寫入。"

    try:
        # 使用 INPUT_SHEET_NAME 連接工作表
        worksheet = gsheets.worksheet(INPUT_SHEET_NAME)
    except gspread.WorksheetNotFound:
        return f"❌ 錯誤：工作表 '{INPUT_SHEET_NAME}' 不存在。"
    except Exception as e:
        return f"❌ 錯誤：無法連接工作表。請檢查權限或名稱: {e}"

    # ✅ 使用 if/else 判斷並標準化 '是否必需' 的值
    if is_essential:
        essential_status = '是'
    else:
        essential_status = '否'

    new_row = [date, category, amount, payment_method, essential_status]
    # ✅ I/O to GoogleSheet (寫入)
    worksheet.append_row(new_row)

    return f"✅ 新增記錄成功：{date}, {category}, ${amount} 元。"

In [39]:
def analyze_and_update():
    """
    從 Google Sheet 讀取數據，執行 Gemini AI 分析，並將報告寫入 '分析報告'。
    """
    if not sheets_enabled:
        return pd.DataFrame(), "❌ Google Sheets 未啟用，無法分析。", "寫入狀態：未啟用。"

    # --- 讀取數據 (✅ I/O to GoogleSheet) ---
    try:
        worksheet = gsheets.worksheet(INPUT_SHEET_NAME)
        # 使用 get_all_values() 讀取所有數據，確保抓取最新資料
        all_data = worksheet.get_all_values()
    except Exception as e:
        return pd.DataFrame(), f"❌ 讀取錯誤: {e}", "寫入狀態：讀取失敗，請確認工作表名稱。"

    if len(all_data) <= 1:
        return pd.DataFrame(), "❌ 試算表是空的，沒有足夠的消費數據可供分析。", "寫入狀態：無數據，未寫入。"

    # 使用讀取到的第一行作為欄位名稱
    df = pd.DataFrame(all_data[1:], columns=all_data[0])

    # --- 基本數據處理與分析 (✅ 使用 for 迴圈) ---

    # 💥 關鍵修正：直接定義程式碼期望的欄位名稱，避免 COLUMN_NAMES 索引錯誤
    AMOUNT_COL = '金額'      # 請確保這個名稱與您的試算表標題完全匹配！
    ESSENTIAL_COL = '是否必需' # 請確保這個名稱與您的試算表標題完全匹配！
    CATEGORY_COL = '品項'     # 請確保這個名稱與您的試算表標題完全匹配！


    # 檢查欄位是否存在，如果不存在，顯示實際讀到的標題供除錯
    if AMOUNT_COL not in df.columns or ESSENTIAL_COL not in df.columns or CATEGORY_COL not in df.columns:
        actual_cols = ", ".join(df.columns.tolist())
        return pd.DataFrame(), f"❌ 欄位錯誤：找不到預期的 '{AMOUNT_COL}', '{ESSENTIAL_COL}', '{CATEGORY_COL}' 欄位。試算表實際標題為：{actual_cols}", "寫入狀態：欄位錯誤。"

    # 強化金額清洗 - 移除貨幣符號、逗號等，再轉數字
    df[AMOUNT_COL] = df[AMOUNT_COL].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df[AMOUNT_COL] = pd.to_numeric(df[AMOUNT_COL], errors='coerce').fillna(0)

    total_spending = df[AMOUNT_COL].sum()
    essential_spending = 0

    # ✅ 使用 for 迴圈統計必需花費
    for index, row in df.iterrows():
        # 使用正確的欄位名稱變數
        if row[ESSENTIAL_COL] == '是':
            essential_spending += row[AMOUNT_COL]

    if total_spending == 0:
        essential_percent = 0.00
    else:
        essential_percent = essential_spending/total_spending*100

    non_essential_spending = total_spending - essential_spending

    # 使用正確的欄位名稱變數
    category_summary = df.groupby(CATEGORY_COL)[AMOUNT_COL].sum().sort_values(ascending=False).reset_index()
    category_summary.rename(columns={AMOUNT_COL: '總花費金額 (元)'}, inplace=True)

    # 建立給 Gemini 的分析提示 (Prompt)
    ai_data_for_prompt = f"""
    [消費數據摘要]
    總消費金額: {total_spending:,.2f} 元
    必需花費總額: {essential_spending:,.2f} 元
    非必需花費總額: {non_essential_spending:,.2f} 元
    必需品佔比: {essential_percent:.2f}%
    品項消費彙總 (前三名):
        {category_summary.head(3).to_string(index=False)}

    請根據這些數據，以專業財務顧問的語氣，撰寫一份 300 字左右的「消費習慣分析報告」。
    報告需包含：1. 財務表現優點、2. 潛在風險警示、3. 一項具體可行預算建議。
    """

    # --- 呼叫 Gemini API 執行 AI 分析 ---
    try:
        if 'client' not in globals():
             raise Exception("Gemini Client 未初始化，請檢查 API Key 設定。")

        ai_prompt = f"請分析以下家庭記帳數據：\n{ai_data_for_prompt}"
        response = client.models.generate_content(
            model=GEMINI_MODEL,
            contents=ai_prompt
        )
        ai_analysis_report = response.text
        ai_status = "✅ AI 報告已由 Gemini 成功生成。"
    except APIError as e:
        ai_analysis_report = f"❌ Gemini API 呼叫失敗。請檢查您的 API Key 或配額。\n錯誤細節: {e}"
        ai_status = "❌ AI 報告生成失敗。"
    except Exception as e:
        ai_analysis_report = f"❌ 發生未知錯誤: {e}"
        ai_status = "❌ AI 報告生成失敗。"


    # --- 寫回 Google Sheet (✅ I/O to GoogleSheet) ---
    try:
        output_worksheet = gsheets.worksheet(OUTPUT_SHEET_NAME)
    except gspread.WorksheetNotFound:
        output_worksheet = gsheets.add_worksheet(title=OUTPUT_SHEET_NAME, rows="200", cols="20")

    output_worksheet.clear()

    # 1. 寫入品項彙總 (A1)
    output_worksheet.update('A1', [category_summary.columns.values.tolist()] + category_summary.values.tolist())

    # 2. 寫入 AI 報告標題與內容
    start_row_ai = len(category_summary) + 3
    output_worksheet.update('A' + str(start_row_ai), [['AI 消費習慣分析報告 (Gemini 驅動)']])

    # 將 AI 報告內容寫入
    output_worksheet.update('A' + str(start_row_ai + 1), [[ai_analysis_report]])


    summary_text = (
        f"**總消費金額：** ${total_spending:,.2f} 元\n"
        f"**必需花費佔比：** {essential_percent:.2f}%\n\n"
        f"**--- AI 智慧分析報告 ---**\n"
        f"{ai_analysis_report}"
    )
    write_status = f"✅ 分析結果與 AI 報告已成功寫入 Google 試算表中的 **{OUTPUT_SHEET_NAME}** 工作表！ ({ai_status})"

    return category_summary, summary_text, write_status

In [40]:
# --- Gradio 介面配置與啟動 (✅ 符合 Gradio 網頁介面要求) ---

# A. 記帳輸入介面
input_tab = gr.Interface(
    fn=add_new_expense,
    inputs=[
        gr.Textbox(label="日期 (例如: 2023/10/26)", value=datetime.now().strftime("%Y/%m/%d")),
        gr.Dropdown(CATEGORIES, label="品項/類別", value='外食'),
        gr.Number(label="金額", minimum=0),
        gr.Radio(PAYMENTS, label="支付方式", value='電子支付'),
        gr.Checkbox(label="是否必需 (勾選為 '是')", value=True)
    ],
    outputs=gr.Markdown(label="寫入狀態"),
    title="新增消費記錄",
    description="在此填寫您的消費資料，資料將直接寫入 Google Sheet 的 '記帳資料' 工作表。"
)

# B. 數據分析介面
output_tab = gr.Interface(
    fn=analyze_and_update,
    inputs=None,
    outputs=[
        gr.Dataframe(label="各品項消費彙總"),
        gr.Markdown(label="主要分析摘要與 AI 報告"),
        gr.Markdown(label="Google Sheet 寫入狀態")
    ],
    title="消費數據分析與報告 (Gemini AI 驅動)",
    description="點擊 Submit 讀取最新數據，執行 Gemini AI 分析，並將報告寫入 '分析報告' 工作表。",
    live=False
)

# 組合兩個介面到一個 Gradio Tabs
accountant_app = gr.TabbedInterface(
    [input_tab, output_tab],
    ["🧾 記帳輸入", "🤖 數據分析與 AI 報告"],
    title="家庭記帳與消費習慣分析工具 (Gemini)"
)

# 啟動 Gradio 介面
accountant_app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bcea292acaf8ff0384.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
